# Setup references

In [ ]:
#r "nuget: Hiperspace"
#r "nuget: Hiperspace.Heap"
#r "./bin/debug/net8.0/CousinProblem.dll"
#r "nuget: Microsoft.DotNet.Interactive.Mermaid, 1.0.0-beta.24229.4"
;;
open Hiperspace
open Hiperspace.Heap
open Cousins
open Microsoft.DotNet.Interactive.Mermaid
;;


# Setup sample data

In [ ]:
open Hiperspace
open Hiperspace.Heap
open Cousins
open Microsoft.DotNet.Interactive.Mermaid

let driver = new HeapSpace()
let space = new CousinsSpace (driver)

let tree = 
  [
    Person (Name = "Eve", Gender = Gender.Female);
    Person (Name = "Jane", Gender = Gender.Female, Mother =  Person (Name = "Eve" ));
    Person (Name = "Jack", Gender = Gender.Male, Mother =  Person (Name = "Eve" ));
    Person (Name = "Adam", Gender = Gender.Male, Father =  Person (Name = "Jack" ));
    Person (Name = "Liz", Gender = Gender.Female, Father =  Person (Name = "Adam" ));
    Person (Name = "Mary", Gender = Gender.Female, Mother =  Person (Name = "Jane" ));
    Person (Name = "Burt", Gender = Gender.Male, Mother=  Person (Name = "Jane" ));
    Person (Name = "Rob", Gender = Gender.Male, Father =  Person (Name = "Burt" ));
    Person (Name = "John", Gender = Gender.Male);
    Person (Name = "Lucy", Gender = Gender.Female, Father =  Person (Name = "John" ), Mother =  Person (Name = "Mary" ));
    Person (Name = "Mark", Gender = Gender.Male, Father =  Person (Name = "John" ), Mother =  Person (Name = "Mary"));
  ]

tree
|> List.iter (fun i -> space.Persons.Bind (i, false) |> ignore)
;;

# simple graph
show the graph with fitect relations

In [ ]:
open Microsoft.DotNet.Interactive.Mermaid
let strip (s : string) = s.Replace("=","")

let nodes = 
    space.Nodes 
    |> Seq.map      (fun n -> $"    {strip n.SKey}(\"{n.Name} ({n.TypeName})\")\n")
    |> Seq.fold     (fun a i  -> a + i) ""
let edges = 
    space.Edges 
    |> Seq.filter   (fun e -> not (e.From = null || e.To = null))
    |> Seq.map      (fun e -> $"    {strip e.From.SKey} -- {e.TypeName} --> {strip e.To.SKey}\n")
    |> Seq.fold     (fun a i -> a + i ) ""

let mermaid = $"""
flowchart TD 
{nodes}
{edges}
"""
MermaidMarkdown (mermaid)


# inferred relations

In [ ]:
let temp = new CousinsSpace(new GenerationSpace ([|new HeapSpace(); space|]))

let relatives (p : Path seq) =
    p
    |> List.ofSeq
    |> List.filter  (fun r -> not (r.From = null || r.To = null || r.TypeName = "relation" ))

space.Persons 
|> List.ofSeq
|> List.map         (fun p -> relatives p.Relatives)
|> List.fold        (fun a i -> i @ a) []
|> List.iter        (fun i -> temp.Paths.Add i |> ignore)

let edges2 = 
    temp.Edges
    |> List.ofSeq
    |> List.filter  (fun r -> not (r.From = null || r.To = null || r.TypeName = "relation" ))
    |> List.map     (fun e -> e, match e.TypeName with | "Mother" | "Father" | "Child" -> "-" | _ -> ".")
    |> List.map     (fun (e, c) -> $"    {strip (e.From.SKey)} {c}- {e.TypeName} {c}-> {strip (e.To.SKey)}\n")
    |> List.fold    (fun a i -> a + i ) ""

let mermaid = $"""
flowchart TD 
{nodes}
{edges2}
"""
MermaidMarkdown (mermaid)
